In [1]:
from nmm.cumulant import cumulant,cum

In [2]:
from qutip import sigmax,sigmay,sigmaz

In [3]:
import numpy as np

In [4]:
type(1.0)

float

In [34]:
cc_c=cum.csolve(T=1.0,alpha=0.05,wc=5.0,t=np.linspace(0,10,1000,dtype=np.float32),eps=1e-6)

In [35]:
type(1.0)

float

In [36]:
cc_c.bose(1.0)

0.5819767068693265

In [37]:
%timeit cc_c.bose(1)

93.4 ns ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [38]:
%timeit cc_c._gamma(3,2,1,5)

514 ns ± 7.66 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [39]:
import numpy as np

In [40]:
help(cc_c)

Help on csolve object:

class csolve(builtins.object)
 |  Methods defined here:
 |
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  __reduce__ = __reduce_cython__(...)
 |
 |  __reduce_cython__(self)
 |
 |  __setstate__ = __setstate_cython__(...)
 |
 |  __setstate_cython__(self, __pyx_state)
 |
 |  bose(self, nu)
 |
 |  gamma(self, w: 'cython.float', w1: 'cython.float', t)
 |
 |  gammafa(self, w: 'cython.float', w1: 'cython.float', t: 'cython.float')
 |
 |  spectral_density(self, w: 'cython.float')
 |
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  eps
 |
 |  --------------------------------------------------------

In [41]:
%timeit cc_c.gamma(1,1,np.linspace(0,10,1000))

/home/mcditoos/anaconda3/envs/qutip-dev/lib/python3.12/site-packages/scipy/integrate/_quadpack_py.py:446: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  re_retval = quad(refunc, a, b, args, full_output, epsabs,


1.06 s ± 48.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
import nmm

In [43]:
bath=nmm.OverdampedBath(lam=0.05,gamma=5,T=1)

In [44]:
cc=cumulant.csolve(Hsys=sigmaz()/2,baths=bath,t=5,Qs=(sigmax()+sigmay()+sigmaz()),eps=1e-6)

In [45]:
%timeit cc_c._gamma(3,1,1,2)

627 ns ± 38.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [46]:
%timeit cc._γ(3,bath,1,1,2)*2*2

52.7 µs ± 7.13 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [47]:
from scipy.integrate import quad_vec,quad
import numpy as np

In [48]:
t=np.linspace(0,10,1000)
w,w1=1,1
%timeit  [quad(cc_c._gamma,0,np.Inf,args=(w, w1, i),epsabs=cc_c.eps,epsrel=cc_c.eps,complex_func=True)[0] for i in t]

1.35 s ± 153 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
%timeit cc_c.gamma(1,1,np.linspace(0,10,1000))

1.19 s ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
%timeit cc.Γgen(bath,w,w1,t)

21 s ± 595 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
21/1.19

17.647058823529413

In [30]:
results=cc.Γgen(bath,w,w1,t)

In [31]:
results_cc= cc_c.gamma(1,1,np.linspace(0,10,1000))

In [33]:
results_cc-results

array([ 0.00000000e+00+0.j, -1.52871103e-05+0.j,  1.23140835e-05+0.j,
        4.25534133e-05+0.j,  4.34213812e-05+0.j,  1.91931181e-06+0.j,
       -9.22284334e-05+0.j, -1.91822713e-04+0.j, -1.93417049e-04+0.j,
       -5.23751212e-05+0.j,  1.60605139e-04+0.j,  3.45235402e-04+0.j,
        4.41350967e-04+0.j,  6.33084378e-05+0.j,  2.30245104e-04+0.j,
        2.68439681e-04+0.j,  2.06302955e-04+0.j,  1.64613329e-04+0.j,
        7.70543435e-05+0.j, -7.21336613e-05+0.j, -1.70541656e-04+0.j,
       -1.81561298e-04+0.j, -1.29878355e-04+0.j,  6.82465213e-05+0.j,
        2.14638077e-04+0.j,  3.07508984e-04+0.j,  2.41989371e-04+0.j,
        1.77118864e-04+0.j,  7.83523408e-04+0.j, -5.07461772e-06+0.j,
        1.16816636e-04+0.j,  3.36600886e-05+0.j, -6.67302708e-05+0.j,
       -9.26698496e-05+0.j, -1.37866711e-04+0.j, -1.74213480e-04+0.j,
       -1.13267145e-04+0.j, -3.46798454e-05+0.j,  7.90624339e-05+0.j,
        1.42711184e-04+0.j,  2.91599786e-04+0.j,  8.37011605e-05+0.j,
        8.80045562e-

In [28]:
17.6/0.133

132.3308270676692

In [15]:
integrals = quad(cc_c._gammare,0,np.Inf,args=(w, w1, t),epsabs=cc.eps,epsrel=cc.eps,limit=2000)[0]

In [23]:
813/48.3

16.832298136645964

In [26]:
690/4.91

140.5295315682281

In [19]:
cc.Γgen(bath,w,w1,t)

(0.3639613853425275+0j)

In [20]:
integrals

0.3639612944696903

In [21]:
cc_c.spectral_density(1)

0.019230769230769232